# Problem Set 3
Author: Stefano Sperti
Date: October 2025

In [ ]:
# %%
import numpy as np
import pyblp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from numpy.random import default_rng
from scipy.optimize import root
pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

# %%
def foc_norm(p, x_t, xi_t, mc_t):
    r = foc_residuals(p, x_t, xi_t, mc_t)
    return np.max(np.abs(r)), r

def ms_mapping_gap(p, x_t, xi_t, mc_t):
    s, dsdp = sim_shares_and_jacobian(p, x_t, xi_t)
    gap = p - (mc_t - s / np.diag(dsdp))  # zero at fixed point
    return np.max(np.abs(gap)), gap

def sim_shares_and_jacobian(p_t, x_t, xi_t):
    """
    Inputs:
    p_t  : (J,) prices in market t
    x_t  : (J,) quality draws for market t
    xi_t : (J,) demand unobservable for market t
    Returns:
    s_mean : (J,) simulated market shares (inside goods)
    dsdp   : (J, J) Jacobian ds_j/dp_k
    Details:
    Individual utility (up to i.i.d. EV1): v_ij = beta1*x_j + beta2_i*sat_j + beta3_i*wir_j + alpha*p_j + xi_j
    s_ij = exp(v_ij) / (1 + sum_k exp(v_ik))  (outside utility normalized to 0)
    Aggregate s_j = E_i[s_ij] via R-draw simulation.
    Derivative under the integral (per draw): ∂s_ij/∂p_k = alpha * s_ij * (1{j=k} - s_ik)
    => dsdp[j,k] = E_i[ ∂s_ij/∂p_k ] (Monte Carlo average).
    """
    # v_i (R,J): random coeffs enter via sat/wir dummies
    v = (beta1 * x_t[None, :]
        + beta2_draws[:, None] * is_sat[None, :]
        + beta3_draws[:, None] * is_wir[None, :]
        + alpha * p_t[None, :]
        + xi_t[None, :])  # shape (R, J)

    exp_v = np.exp(v - np.max(v, axis=1, keepdims=True))  # safe stab.
    denom = 1.0 + np.sum(exp_v, axis=1, keepdims=True)    # add outside option
    s_ind = exp_v / denom                                 # (R, J)
    s_mean = s_ind.mean(axis=0)                           # (J,)

    # Jacobian: ds_j/dp_k = E_i[ alpha * s_ij * (1{j=k} - s_ik) ]
    dsdp = np.empty((J, J))
    for k in range(J):
        factor = alpha * (np.eye(J)[k][None, :] - s_ind)  
        dsdp[:, k] = np.mean(s_ind * factor, axis=0)
    return s_mean, dsdp

def foc_residuals(p_t, x_t, xi_t, mc_t):
    s, dsdp = sim_shares_and_jacobian(p_t, x_t, xi_t)
    # Single-product FOCs: (p_j - mc_j)*ds_j/dp_j + s_j = 0  for each j
    diag = np.diag(dsdp)  
    return (p_t - mc_t) * diag + s

def solve_equilibrium_root(x_t, xi_t, mc_t, p0=None, tol=1e-12):
    # Root-finding on the J FOCs
    if p0 is None:
        p0 = mc_t + 1.0  # mild markup starting point
    sol = root(lambda p: foc_residuals(p, x_t, xi_t, mc_t), p0, method='hybr', tol=tol)
    return sol.x, sol.success, sol.nfev

# ------------------------------------------------------
# Morrow–Skerlos Fixed-Point (MSFP) mapping per market
# ------------------------------------------------------
def msfp_prices(x_t, xi_t, mc_t, p0=None, max_iter=5000, tol=1e-10, damp=0.8):
    """
    p_{n+1} = mc - diag(dsdp(p_n))^{-1} s(p_n)
    Optional damping for stability.
    """
    if p0 is None:
        p = mc_t + 1.0
    else:
        p = p0.copy()
    for it in range(max_iter):
        s, dsdp = sim_shares_and_jacobian(p, x_t, xi_t)
        diag = np.diag(dsdp)
        # Guard: own-prices derivatives should be negative
        if np.any(diag >= 0):
            # If happens, try to bail with small step toward mc
            p = 0.5 * (p + mc_t)
            continue
        p_new = mc_t - s / diag
        # damping
        p_next = damp * p_new + (1.0 - damp) * p
        # convergence
        if np.max(np.abs(p_next - p)) < tol:
            return p_next, True, it + 1
        p = p_next
    return p, False, max_iter



rng = default_rng(12345)  # reproducible
T = 600                    # markets
J = 4                      
R = 100              
# Demand parameters
beta1 = 1.0                
alpha = -2.0               
# random coefficients: beta2_i ~ N(4,1) on satellite, beta3_i ~ N(4,1) on wired
mu_sat, sd_sat = 4.0, 1.0
mu_wir, sd_wir = 4.0, 1.0
# Cost parameters
gamma0 = 0.5
gamma1 = 0.25
# Correlated unobservables (xi, omega): N(0,0; 1, 0.25; 0.25, 1)
Sigma = np.array([[1.0, 0.25],
                [0.25, 1.0]])
# Product identities: j=0,1 are satellite; j=2,3 are wired
is_sat = np.array([1, 1, 0, 0], dtype=int)
is_wir = 1 - is_sat

# Exogenous characteristics and cost shifter: abs(N(0,1))
x = np.abs(rng.standard_normal((T, J)))
w = np.abs(rng.standard_normal((T, J)))

z = rng.multivariate_normal(mean=np.zeros(2), cov=Sigma, size=T*J)
xi = z[:, 0].reshape(T, J)
omega = z[:, 1].reshape(T, J)

# Marginal costs: ln mc_jt = gamma0 + gamma1 * w_jt + omega_jt/8
mc = np.exp(gamma0 + gamma1 * w + omega / 8.0)  # shape (T, J)

beta2_draws = mu_sat + sd_sat * rng.standard_normal(R)   # satellite taste
beta3_draws = mu_wir + sd_wir * rng.standard_normal(R)   # wired taste



# -------------------------
prices_root = np.empty((T, J))
succ_root   = np.zeros(T, dtype=bool)
evals_root  = np.zeros(T, dtype=int)

prices_msfp = np.empty((T, J))
succ_msfp   = np.zeros(T, dtype=bool)
iters_msfp  = np.zeros(T, dtype=int)

for t in range(T):
    # Root-solver

    # MSFP, warm-start at root solution (or mc+1 if root failed)
    p0 = (mc[t] + 1.0)
    p_ms, ok_ms, it_ms = msfp_prices(x[t], xi[t], mc[t], p0=p0, max_iter=2000, tol=1e-12, damp=0.85)
    prices_msfp[t] = p_ms
    succ_msfp[t] = ok_ms
    iters_msfp[t] = it_ms

    p_star, ok, nfev = solve_equilibrium_root(x[t], xi[t], mc[t], p0=p_ms)
    prices_root[t] = p_star
    succ_root[t] = ok
    evals_root[t] = nfev


# -------------------------------------------
shares_root = np.empty((T, J))
shares_msfp = np.empty((T, J))
for t in range(T):
    s_r, _ = sim_shares_and_jacobian(prices_root[t], x[t], xi[t])
    s_m, _ = sim_shares_and_jacobian(prices_msfp[t], x[t], xi[t])
    shares_root[t] = s_r
    shares_msfp[t] = s_m

# -------------------------
print(f"Root-solver success: {succ_root.mean():.3f} of markets")
print(f"MSFP success:        {succ_msfp.mean():.3f} of markets")

# Compare the two methods (they should match very closely if both converged)
diff = np.abs(prices_root - prices_msfp)
print("Max |price_root - price_msfp|:", np.nanmax(diff))

t = 0  # pick a market to inspect
p_r = prices_root[t]
p_m = prices_msfp[t]

fn_r, rvec_r = foc_norm(p_r, x[t], xi[t], mc[t])
fn_m, rvec_m = foc_norm(p_m, x[t], xi[t], mc[t])
mg_r, gvec_r = ms_mapping_gap(p_r, x[t], xi[t], mc[t])
mg_m, gvec_m = ms_mapping_gap(p_m, x[t], xi[t], mc[t])

print("Root solution:  max FOC residual =", fn_r, " | max MS gap =", mg_r)
print("MSFP solution:  max FOC residual =", fn_m, " | max MS gap =", mg_m)
print("max |p_root - p_msfp| =", np.max(np.abs(p_r - p_m)))

# Optional: assemble a tidy DataFrame to export
records = []
for t in range(T):
    for j in range(J):
        nest_ids = 1 if (j + 1 ) == 1 or (j + 1) == 0 else 1
        records.append({
            "market_ids": t+1,
            "product_ids": j+1,
            "nesting_ids": nest_ids,
            "is_satellite": is_sat[j],
            "is_wired": is_wir[j],
            "x": x[t, j],
            "w": w[t, j],
            "xi": xi[t, j],
            "omega": omega[t, j],
            "mc": mc[t, j],
            "prices": prices_root[t, j],
            "shares": shares_root[t, j],
        })
df = pd.DataFrame.from_records(records)
df.to_csv("paytv_sim_equilibrium.csv", index=False)
print(df.head())

Root-solver success: 1.000 of markets
MSFP success:        1.000 of markets
Max |price_root - price_msfp|: 2.318145675417327e-13
Root solution:  max FOC residual = 5.551115123125783e-17  | max MS gap = 4.440892098500626e-16
MSFP solution:  max FOC residual = 7.210898544940392e-14  | max MS gap = 1.6120438317557273e-13
max |p_root - p_msfp| = 1.6120438317557273e-13
   market_ids  product_ids  nesting_ids  is_satellite  is_wired         x  \
0           1            1            1             1         0  1.423825   
1           1            2            1             1         0  1.263728   
2           1            3            1             0         1  0.870662   
3           1            4            1             0         1  0.259173   
4           2            1            1             1         0  0.075343   

          w        xi     omega        mc    prices    shares  
0  1.065850 -0.578893  0.554603  2.306631  2.830617  0.042903  
1  0.309725  1.305041 -0.000393  1.781369 

In [ ]:
# ===== Build & estimate the BLP problem from your simulated data =====
import numpy as np
import pandas as pd
import pyblp
import scipy.sparse as sp

# --- 1) Prep product_data in the format pyBLP expects ---
# Your df currently has: market_ids, product_ids, is_satellite, is_wired, x, w, xi, omega, mc, prices, shares
prod = df.copy()

# Single-product firms → each product its own firm
prod["firm_ids"] = prod["product_ids"]

# Rename to match variable names we’ll refer to in formulations
prod = prod.rename(columns={
    "is_satellite": "satellite",
    "is_wired": "wired",
    "prices": "prices",
    "shares": "shares",
    "x": "x",
    "w": "w",
})

# (Optional but good): make sure types are numeric
for c in ["market_ids", "product_ids", "firm_ids", "satellite", "wired", "x", "w", "prices", "shares"]:
    prod[c] = pd.to_numeric(prod[c])

# Sanity: shares must be in (0,1) and sum to < 1 per market (outside share = 1 - sum inside)
inside_sum = prod.groupby("market_ids")["shares"].sum()
assert np.all(inside_sum < 1.0 + 1e-8), "Inside shares must sum to < 1 in each market."

# --- 2) Formulations (match your DGP) ---
form_X1 = pyblp.Formulation("0 + prices + x + satellite")
form_X2 = pyblp.Formulation("1 + w + omega")
form_X3 = pyblp.Formulation("1 + w")

# Integration for simulated agents (product-level rule is a good default)
integration = pyblp.Integration(
    "product",
    size=20
)

# --- 3) Helper to attach instrument matrices as numbered columns ---
def attach_instruments(df, Z, prefix):
    if sp.issparse(Z):
        Z = Z.toarray()
    else:
        Z = np.asarray(Z)
    for k in range(Z.shape[1]):
        df[f"{prefix}{k}"] = Z[:, k]

# --- 4) Demand-only estimation (build BLP-style IVs) ---
prod_A = prod.copy()
Z_D = pyblp.build_blp_instruments(pyblp.Formulation("0 + x + satellite + wired"), prod_A)
attach_instruments(prod_A, Z_D, "demand_instruments")

problem_A = pyblp.Problem(
    product_formulations=(form_X1, form_X2, None),  # demand only
    product_data=prod_A,
    integration=integration
)
results_A = problem_A.solve()   # default GMM

# --- 5) Joint demand + supply (add supply IVs; e.g., sums of w) ---
prod_B = prod.copy()
attach_instruments(prod_B, Z_D, "demand_instruments")
Z_S = pyblp.build_blp_instruments(pyblp.Formulation("0 + w"), prod_B)
attach_instruments(prod_B, Z_S, "supply_instruments")

problem_B = pyblp.Problem(
    product_formulations=(form_X1, form_X2, form_X3),
    product_data=prod_B,
    integration=integration,
    costs_type="log"   # matches your mc specification
)
results_B = problem_B.solve()

# --- 6) “Optimal IVs”: compute feasible optimal instruments & re-estimate ---
opt = results_B.compute_optimal_instruments()
problem_C = opt.to_problem()
results_C = problem_C.solve()

# --- 7) Summaries: parameters, elasticities, diversion ---
def demand_table(res, label):
    return pd.DataFrame({
        "spec": label,
        "param": ["alpha(price)", "beta_x", "beta_sat", "beta_wired"],
        "estimate": res.beta.ravel(),
        "std_error": res.beta_se.ravel()
    })

tab = pd.concat([
    demand_table(results_A, "Demand only"),
    demand_table(results_B, "Joint D+S"),
    demand_table(results_C, "Joint + optimal IV"),
], ignore_index=True)

print("\n=== Demand parameter estimates (SEs) ===")
print(tab.round(4).to_string(index=False))

# Elasticities & diversion (use preferred = optimal IVs)
preferred = results_C
elast_est = preferred.compute_elasticities()
div_est   = preferred.compute_diversion_ratios()

# True elasticities/diversion at your simulated equilibrium (optional, for comparison)
# Provide the true params in the same order as form_X1 and form_X2
beta_true  = np.array([-2.0, 1.0, 4.0, 4.0])       # [alpha, beta_x, mean_sat, mean_wired]
Sigma_true = np.diag([1.0, 1.0])                    # RC stds on [sat, wired]
gamma_true = np.array([0.5, 0.25])                  # [gamma0, gamma1]

problem_true = pyblp.Problem(
    product_formulations=(form_X1, form_X2, form_X3),
    product_data=prod,
    integration=integration,
    costs_type="log"
)
elast_true = problem_true.compute_elasticities(beta=beta_true, sigma=Sigma_true, gamma=gamma_true)
div_true   = problem_true.compute_diversion_ratios(beta=beta_true, sigma=Sigma_true, gamma=gamma_true)

# Average own-price elasticity across markets/products
def avg_own(elast, mkts, J=4):
    owns = []
    for t in np.unique(mkts):
        idx = np.where(mkts == t)[0]
        E = elast[np.ix_(idx, idx)]
        owns.extend(np.diag(E))
    return float(np.mean(owns))

mkts = prod["market_ids"].values
own_true = avg_own(elast_true, mkts)
own_est  = avg_own(elast_est,  mkts)
print("\n=== Avg own-price elasticity ===")
print(pd.DataFrame({"metric":["avg own-price elasticity"], "true":[own_true], "estimated":[own_est]}).round(4).to_string(index=False))

# Average diversion matrices (JxJ) across markets
def avg_block(mat, mkts, J=4):
    mats = []
    for t in np.unique(mkts):
        idx = np.where(mkts == t)[0]
        mats.append(mat[np.ix_(idx, idx)])
    return np.mean(mats, axis=0)

avg_div_true = avg_block(div_true, mkts)
avg_div_est  = avg_block(div_est,  mkts)

def to_tbl(M, name):
    d = []
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            d.append({"from": i+1, "to": j+1, name: M[i,j]})
    return pd.DataFrame(d)

print("\n=== Avg diversion matrix (TRUE) ===")
print(pd.pivot_table(to_tbl(avg_div_true, "v"), index="from", columns="to", values="v").round(4))
print("\n=== Avg diversion matrix (ESTIMATED) ===")
print(pd.pivot_table(to_tbl(avg_div_est,  "v"), index="from", columns="to", values="v").round(4))



Detected collinearity issues with [demand_instruments0, demand_instruments1, demand_instruments2, satellite, wired] and at least one other column in ZD. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.


ValueError: sigma should be specified only when X2 was formulated.

In [ ]:


# Convert robustly to a pandas DataFrame
if isinstance(raw_pd, pd.DataFrame):
    prod_sim = raw_pd.copy()
else:
    # handles numpy structured arrays
    prod_sim = pd.DataFrame({name: raw_pd[name] for name in raw_pd.dtype.names})

# Helper: attach instrument matrix as numbered columns
def attach_instruments(df: pd.DataFrame, Z, prefix: str):
    # ensure dense 2D ndarray
    if sp.issparse(Z):
        Z = Z.toarray()
    else:
        Z = np.asarray(Z)
    for k in range(Z.shape[1]):
        df[f'{prefix}{k}'] = Z[:, k]

# Build demand IVs (BLP sums)
Z_D = pyblp.build_blp_instruments(
    pyblp.Formulation('0 + x + satellite + wired'),
    prod_sim
)

prod_A = prod_sim.copy()
attach_instruments(prod_A, Z_D, 'demand_instruments')

problem_A = pyblp.Problem(
    product_formulations=(form_X1, form_X2, None),
    product_data=prod_A,
    integration=integration
)
results_A = problem_A.solve()


# ----------------------------
# 4) Estimation: instruments & problems
# ----------------------------
# Helper to attach instrument matrices as numbered columns
def attach_instruments(df, Z, prefix):
    for k in range(Z.shape[1]):
        df[f'{prefix}{k}'] = Z[:, k]

# (A) Demand-only: build BLP-style demand IVs (sums of rivals' characteristics)
prod_A = prod_sim.copy()
Z_D = pyblp.build_blp_instruments(pyblp.Formulation('0 + x + satellite + wired'), prod_A)
attach_instruments(prod_A, Z_D, 'demand_instruments')

problem_A = pyblp.Problem(
    product_formulations=(form_X1, form_X2, None),  # no supply side
    product_data=prod_A,
    integration=integration
)
results_A = problem_A.solve()

# (B) Joint demand + supply: add supply IVs (e.g., sums of w)
prod_B = prod_sim.copy()
attach_instruments(prod_B, Z_D, 'demand_instruments')
Z_S = pyblp.build_blp_instruments(pyblp.Formulation('0 + w'), prod_B)
attach_instruments(prod_B, Z_S, 'supply_instruments')

problem_B = pyblp.Problem(
    product_formulations=(form_X1, form_X2, form_X3),
    product_data=prod_B,
    integration=integration,
    costs_type='log'
)
results_B = problem_B.solve()

# (C) Optimal IVs (feasible optimal instruments computed from results_B)
opt = results_B.compute_optimal_instruments()
problem_C = opt.to_problem()
results_C = problem_C.solve()

# ----------------------------
# 5) Tables: parameters, elasticities, diversion
# ----------------------------
def demand_table(res, label):
    cols = ['alpha(price)', 'beta_x', 'beta_sat', 'beta_wired']
    return pd.DataFrame({
        'spec': label,
        'param': cols,
        'estimate': res.beta.ravel(),   # matches X1 order
        'std_error': res.beta_se.ravel()
    })

tab = pd.concat([
    demand_table(results_A, 'Demand only'),
    demand_table(results_B, 'Joint D+S'),
    demand_table(results_C, 'Joint + optimal IV')
], ignore_index=True)

# True vs estimated elasticities/diversion (use preferred = results_C by default)
preferred = results_C

# True objects at the simulated equilibrium (use the known params)
problem_true = pyblp.Problem(
    product_formulations=(form_X1, form_X2, form_X3),
    product_data=prod_sim,
    integration=integration,
    costs_type='log'
)

elast_true = problem_true.compute_elasticities(beta=beta_true, sigma=Sigma_true, gamma=gamma_true)
div_true   = problem_true.compute_diversion_ratios(beta=beta_true, sigma=Sigma_true, gamma=gamma_true)

elast_est = preferred.compute_elasticities()
div_est   = preferred.compute_diversion_ratios()

def avg_own_price(elast, mkts):
    owns = []
    for t in np.unique(mkts):
        idx = np.where(mkts == t)[0]
        E = elast[np.ix_(idx, idx)]
        owns.extend(np.diag(E))
    return float(np.mean(owns))

own_compare = pd.DataFrame({
    'metric': ['avg own-price elasticity'],
    'true': [avg_own_price(elast_true, prod_sim['market_ids'].values)],
    'estimated': [avg_own_price(elast_est, prod_sim['market_ids'].values)]
})

def avg_market_matrix(mat, mkts, J):
    blocks = []
    for t in np.unique(mkts):
        idx = np.where(mkts == t)[0]
        blocks.append(mat[np.ix_(idx, idx)])
    return np.mean(blocks, axis=0)

avg_div_true = avg_market_matrix(div_true, prod_sim['market_ids'].values, J)
avg_div_est  = avg_market_matrix(div_est,  prod_sim['market_ids'].values, J)

# Tidy diversion tables (average across markets)
def mat_to_df(M, name):
    rows = []
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            rows.append({'from_product': i+1, 'to_product': j+1, name: M[i, j]})
    return pd.DataFrame(rows)

diversion_compare = mat_to_df(avg_div_true, 'div_true').merge(
    mat_to_df(avg_div_est, 'div_est'),
    on=['from_product','to_product']
).sort_values(['from_product','to_product'])

# ----------------------------
# 6) Print quick results
# ----------------------------
pd.set_option('display.width', 120)
print("\n=== Demand parameter estimates (SEs) ===")
print(tab.round(4).to_string(index=False))

print("\n=== Average own-price elasticity (true vs. estimated) ===")
print(own_compare.round(4).to_string(index=False))

print("\n=== Avg diversion matrix across markets (TRUE) ===")
print(pd.pivot_table(mat_to_df(avg_div_true, 'v'), index='from_product', columns='to_product', values='v').round(4))

print("\n=== Avg diversion matrix across markets (ESTIMATED) ===")
print(pd.pivot_table(mat_to_df(avg_div_est, 'v'), index='from_product', columns='to_product', values='v').round(4))


In [ ]:
def derivative_share_matrix(beta1, mu2, sigma2, mu3, sigma3, xi, x, J, T, prices):
    N = 100
    beta2_simulated = np.random.normal(mu2, sigma2, N)
    beta3_simulated = np.random.normal(mu3, sigma3, N)
    share_matrix_derivatives = np.zeros((J, J, T))
    for j in range(J):
        for m in range(J):
            for t in range(T):
                if j == m:
                    derivative = 0
                    for n in range(N):
                            exp_utilities = np.exp(beta1 * x[j, t] + beta2_simulated[n]* ((j ==0)|(j==1)) + beta3_simulated[n]*((j==2)|(j==3)) + xi[j,t] + alpha * prices[t, j])
                            sum_exp_utilities = np.sum(exp_utilities, axis=0, keepdims=True)
                            probabilities = exp_utilities / (1 + sum_exp_utilities)
                            derivative += probabilities * (1 - probabilities)
                    derivative_mean = derivative / N
                else:
                    for n in range(N):
                        exp_utilities_jt  = np.exp(beta1 * x[j, t] + beta2_simulated[n]* ((j ==0)|(j==1)) + beta3_simulated[n]*((j==2)|(j==3)) + xi[j,t] + alpha * prices[t, j])
                        exp_utilities_mt = np.exp(beta1 * x[m, t] + beta2_simulated[n]* ((m ==0)|(m==1)) + beta3_simulated[n]*((m==2)|(m==3)) + xi[m,t] + alpha * prices[t, m])
                        sum_exp_utilities = np.sum(exp_utilities, axis=0, keepdims=True)
                        probabilities_jt = exp_utilities_jt / (1 + sum_exp_utilities)
                        probabilities_mt = exp_utilities_mt / (1 + sum_exp_utilities)
                        derivative += -probabilities_jt * probabilities_mt   
                    derivative_mean = derivative / N
                share_matrix_derivatives[j, m, t] = derivative_mean


In [ ]:
share_matrix = np.empty((T, J))
for t in range(T):
    for j in range(J):
        exp_utilities = np.exp(beta1 * x[t, j] + mu2 * ((j ==0)|(j==1)) + mu3 * ((j==2)|(j==3)) + xi[t,j])
        sum_exp_utilities = np.sum(exp_utilities)
        share_matrix[t, j] = exp_utilities / (1 + sum_exp_utilities)
share_matrix_derivatives = derivative_share_matrix(beta1, mu2, sigma2, mu3, sigma3, xi.T, x.T, J, T, prices=np.random.normal(5,1,(T,J)))

### Simulate data with pyblp

In [5]:
import numpy as np
import pandas as pd
import pyblp

pyblp.options.verbose = False         # reduce pyBLP chatter
pyblp.options.digits = 3              # pretty printing digits
rng = np.random.default_rng(2025)     # reproducible RNG

# ============================================================
# E1. DATA GENERATION VIA pyBLP.Simulation
# ------------------------------------------------------------
# Conventions:
#   Utility: δ_j = β_x * x_j  – α * p_j  + ξ_j, with α > 0
#   => price coefficient in utility is -α
#   We target -2.0 as requested, so set α = +2.0 (alpha_mag).
# ============================================================

T = 600                  # markets
J = 4                    # products per market (4 single-product firms)
alpha_mag = 2.0          # price coefficient in utility is -2.0
beta_x = 1.0             # true coefficient on x
rho_true = 0.55          # nested-logit correlation
c0, c1 = 5.0, 1.0        # linear cost: mc = c0 + c1 * w + ω
corr_xi_omega = 0.25     # corr(ξ, ω)

print("============================================================")
print("EXERCISE 1 —  with pyBLP.Simulation")
print("------------------------------------------------------------")
id_data = pyblp.build_id_data(T=T, J=J, F=4)
base = pd.DataFrame(pyblp.data_to_dict(id_data))   
# Ownership and nests: 1..4 single-product firms; nests: 0(wired) for 1&2, 1(sat) for 3&4
base["firm_ids"] = np.tile(np.array([1, 2, 3, 4]), T)
base["nesting_ids"] = np.tile(np.array([0, 0, 1, 1]), T)

# Exogenous shifters: |N(0,1)| for x (demand) and w (cost)
base["x"] = np.abs(rng.normal(size=len(base)))
base["w"] = np.abs(rng.normal(size=len(base)))

# Formulations: X1(demand), X2(None=logit/nested), X3(supply)
form_demand = pyblp.Formulation("1 + prices + x")  # constant, price, x
form_supply = pyblp.Formulation("1 + w")
formulations = (form_demand, None, form_supply)

# Configure the simulation (true parameters)
simulation = pyblp.Simulation(
    product_formulations=formulations,
    product_data=base,
    beta=[0.0, -alpha_mag, beta_x],   # intercept, price, x  (price coef is -alpha_mag = -2.0)
    gamma=[c0, c1],                    # cost side: constant and w
    rho=rho_true,                      # nested logit
    xi_variance=1.0, omega_variance=1.0, correlation=corr_xi_omega,
    costs_type="linear",
    seed=1234
)

# Compute equilibrium-consistent prices & shares
sim_res = simulation.replace_endogenous()
product_data = pd.DataFrame(pyblp.data_to_dict(sim_res.product_data))

# Within-market product index (0..J-1) for summary tables
product_data["position"] = product_data.groupby("market_ids").cumcount()

# Preview: first market (avoid columns that might not exist in older pyBLP)
print("Simulated price/share preview (first market):")
m0 = product_data.query("market_ids == 1").copy()
cols = [c for c in ["market_ids", "position", "firm_ids", "nesting_ids", "x", "w", "prices", "shares"] if c in m0.columns]
print(m0[cols].round(4).to_string(index=False))

EXERCISE 1 —  with pyBLP.Simulation
------------------------------------------------------------
Simulated price/share preview (first market):
market_ids  position firm_ids nesting_ids      x      w  prices  shares
         1         0        1           0 2.4419 0.2274  5.8336  0.0003
         1         1        2           0 0.7654 1.6747  6.4671  0.0000
         1         2        3           1 0.7597 0.1034  4.7754  0.0001
         1         3        4           1 0.2670 3.0356  8.0942  0.0000


In [6]:


# Convert robustly to a pandas DataFrame
if isinstance(raw_pd, pd.DataFrame):
    prod_sim = raw_pd.copy()
else:
    # handles numpy structured arrays
    prod_sim = pd.DataFrame({name: raw_pd[name] for name in raw_pd.dtype.names})

# Helper: attach instrument matrix as numbered columns
def attach_instruments(df: pd.DataFrame, Z, prefix: str):
    # ensure dense 2D ndarray
    if sp.issparse(Z):
        Z = Z.toarray()
    else:
        Z = np.asarray(Z)
    for k in range(Z.shape[1]):
        df[f'{prefix}{k}'] = Z[:, k]

# Build demand IVs (BLP sums)
Z_D = pyblp.build_blp_instruments(
    pyblp.Formulation('0 + x + satellite + wired'),
    prod_sim
)

prod_A = prod_sim.copy()
attach_instruments(prod_A, Z_D, 'demand_instruments')

problem_A = pyblp.Problem(
    product_formulations=(form_X1, form_X2, None),
    product_data=prod_A,
    integration=integration
)
results_A = problem_A.solve()


# ----------------------------
# 4) Estimation: instruments & problems
# ----------------------------
# Helper to attach instrument matrices as numbered columns
def attach_instruments(df, Z, prefix):
    for k in range(Z.shape[1]):
        df[f'{prefix}{k}'] = Z[:, k]

# (A) Demand-only: build BLP-style demand IVs (sums of rivals' characteristics)
prod_A = prod_sim.copy()
Z_D = pyblp.build_blp_instruments(pyblp.Formulation('0 + x + satellite + wired'), prod_A)
attach_instruments(prod_A, Z_D, 'demand_instruments')

problem_A = pyblp.Problem(
    product_formulations=(form_X1, form_X2, None),  # no supply side
    product_data=prod_A,
    integration=integration
)
results_A = problem_A.solve()

# (B) Joint demand + supply: add supply IVs (e.g., sums of w)
prod_B = prod_sim.copy()
attach_instruments(prod_B, Z_D, 'demand_instruments')
Z_S = pyblp.build_blp_instruments(pyblp.Formulation('0 + w'), prod_B)
attach_instruments(prod_B, Z_S, 'supply_instruments')

problem_B = pyblp.Problem(
    product_formulations=(form_X1, form_X2, form_X3),
    product_data=prod_B,
    integration=integration,
    costs_type='log'
)
results_B = problem_B.solve()

# (C) Optimal IVs (feasible optimal instruments computed from results_B)
opt = results_B.compute_optimal_instruments()
problem_C = opt.to_problem()
results_C = problem_C.solve()

# ----------------------------
# 5) Tables: parameters, elasticities, diversion
# ----------------------------
def demand_table(res, label):
    cols = ['alpha(price)', 'beta_x', 'beta_sat', 'beta_wired']
    return pd.DataFrame({
        'spec': label,
        'param': cols,
        'estimate': res.beta.ravel(),   # matches X1 order
        'std_error': res.beta_se.ravel()
    })

tab = pd.concat([
    demand_table(results_A, 'Demand only'),
    demand_table(results_B, 'Joint D+S'),
    demand_table(results_C, 'Joint + optimal IV')
], ignore_index=True)

# True vs estimated elasticities/diversion (use preferred = results_C by default)
preferred = results_C

# True objects at the simulated equilibrium (use the known params)
problem_true = pyblp.Problem(
    product_formulations=(form_X1, form_X2, form_X3),
    product_data=prod_sim,
    integration=integration,
    costs_type='log'
)

elast_true = problem_true.compute_elasticities(beta=beta_true, sigma=Sigma_true, gamma=gamma_true)
div_true   = problem_true.compute_diversion_ratios(beta=beta_true, sigma=Sigma_true, gamma=gamma_true)

elast_est = preferred.compute_elasticities()
div_est   = preferred.compute_diversion_ratios()

def avg_own_price(elast, mkts):
    owns = []
    for t in np.unique(mkts):
        idx = np.where(mkts == t)[0]
        E = elast[np.ix_(idx, idx)]
        owns.extend(np.diag(E))
    return float(np.mean(owns))

own_compare = pd.DataFrame({
    'metric': ['avg own-price elasticity'],
    'true': [avg_own_price(elast_true, prod_sim['market_ids'].values)],
    'estimated': [avg_own_price(elast_est, prod_sim['market_ids'].values)]
})

def avg_market_matrix(mat, mkts, J):
    blocks = []
    for t in np.unique(mkts):
        idx = np.where(mkts == t)[0]
        blocks.append(mat[np.ix_(idx, idx)])
    return np.mean(blocks, axis=0)

avg_div_true = avg_market_matrix(div_true, prod_sim['market_ids'].values, J)
avg_div_est  = avg_market_matrix(div_est,  prod_sim['market_ids'].values, J)

# Tidy diversion tables (average across markets)
def mat_to_df(M, name):
    rows = []
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            rows.append({'from_product': i+1, 'to_product': j+1, name: M[i, j]})
    return pd.DataFrame(rows)

diversion_compare = mat_to_df(avg_div_true, 'div_true').merge(
    mat_to_df(avg_div_est, 'div_est'),
    on=['from_product','to_product']
).sort_values(['from_product','to_product'])

# ----------------------------
# 6) Print quick results
# ----------------------------
pd.set_option('display.width', 120)
print("\n=== Demand parameter estimates (SEs) ===")
print(tab.round(4).to_string(index=False))

print("\n=== Average own-price elasticity (true vs. estimated) ===")
print(own_compare.round(4).to_string(index=False))

print("\n=== Avg diversion matrix across markets (TRUE) ===")
print(pd.pivot_table(mat_to_df(avg_div_true, 'v'), index='from_product', columns='to_product', values='v').round(4))

print("\n=== Avg diversion matrix across markets (ESTIMATED) ===")
print(pd.pivot_table(mat_to_df(avg_div_est, 'v'), index='from_product', columns='to_product', values='v').round(4))


ValueError: Per-column arrays must each be 1-dimensional